In [1]:
# Generate new samplelist

In [2]:
import pickle
from pandas import read_csv

In [5]:
# load csv
f = read_csv("../out.csv", sep=' ')
ra = f['RAJ2000'] # RA
dec = f['DEJ2000'] # DEC
label = f['A']
redshift = f['z']
snvss = f['SNVSS']

In [4]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
def gen_rename(ra, dec):
    # get params
    temp_c = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    # Coordinate transform
    ra_rms = tuple(temp_c.ra.hms)
    dec_dms = tuple(temp_c.dec.dms)
    # save name
    ra_h = "%02d" % (int(ra_rms[0]))
    ra_m = "%02d" % (int(ra_rms[1]))
    ra_s_i = np.fix(np.round(ra_rms[2]*100)/100)
    ra_s_f = np.round(ra_rms[2]*100)/100 - ra_s_i
    ra_s = "%02d.%02d" % (int(ra_s_i),int(ra_s_f*100))
    if dec_dms[0] > 0:
        de_d = "+%02d" % (int(dec_dms[0]))
    else:
        de_d = "-%02d" % (abs(int(dec_dms[0])))
    de_m = "%02d" % (abs(int(dec_dms[1])))
    de_s_i = np.fix(np.abs(np.round(dec_dms[2]*10)/10))
    de_s_f = np.abs(np.round(dec_dms[2]*10)/10) - de_s_i
    de_s = "%02d.%01d" % (int(de_s_i),np.round(de_s_f*10))
    rename = 'J' + ''.join([ra_h,ra_m,ra_s,de_d,de_m,de_s])
    
    return rename

In [15]:
# Generate labels
sample_best = {"name":[], "label": [], "redshift":[], "snvss":[]}
for i in range(len(ra)):
    fname = gen_rename(ra=ra[i],dec=dec[i])
    sample_best["name"].append(fname)
    sample_best["label"].append(label[i])
    sample_best["redshift"].append(float(redshift[i]))
    sample_best['snvss'].append(float(snvss[i]))

In [16]:
fname = "../sample-list-cross-match.csv"
samples = read_csv(fname, sep=" ")
# Count matched samples
sample_match = samples["Match"]
sample_idx = samples["Index"]
match_idx = np.where(np.array(sample_match) != "nomatching")[0]

name = np.array(sample_best["name"])
label = np.array(sample_best["label"])
redshift = np.array(sample_best["redshift"])
snvss = np.array(sample_best["snvss"])

In [17]:
# Discard cross matched
name_new = np.delete(name,np.array(sample_idx[match_idx]).astype(int))
label_new = np.delete(label,np.array(sample_idx[match_idx]).astype(int))
redshift_new = np.delete(redshift,np.array(sample_idx[match_idx]).astype(int))
snvss_new = np.delete(snvss,np.array(sample_idx[match_idx]).astype(int))

In [27]:
# add redshift restriction
z_max = 0.3
idx_z = np.where(redshift_new <= z_max)[0]
# remove
name_z = name_new[idx_z]
label_z = label_new[idx_z]
redshift_z = redshift_new[idx_z]
snvss_z = snvss_new[idx_z]

In [28]:
name_new.shape

(17609,)

In [29]:
name_z.shape

(11671,)

In [30]:
label_z.shape

(11671,)

In [31]:
# save
sample_bkp = {"name": list(name_z), "label": list(label_z),
              "redshift":list(redshift_z), "snvss": list(snvss_z)}

In [32]:
with open("../sample-best-cross-discard-redshift.pkl", 'wb') as fp:
    pickle.dump(sample_bkp, fp)